In [ ]:
import pandas as pd

def compute_monotonicity(rankings, node_count):
    """
    计算单调性函数 M(R)
    
    :param rankings: 排名列表的字典，其中键为排名方法名，值为节点排名字典
    :param node_count: 网络中节点的总数
    :return: 单调性函数的值
    """
    total_sum = 0
    
    for ranks in rankings.values():
        rank_counts = pd.Series(ranks).value_counts()
        for count in rank_counts:
            total_sum += count * (count - 1)
    
    norm_factor = node_count * (node_count - 1)
    average_sum = total_sum / norm_factor
    monotonicity_value = (1 - average_sum) ** 2
    
    return monotonicity_value

# 读取排序结果并计算排名，确保相同的Centrality值有相同的排名
def load_and_rank(filename):
    df = pd.read_csv(filename)
    df['Rank'] = df['Centrality'].rank(ascending=False, method='min')  # 使用 rank 确保相同值有相同排名
    rankings = {node: rank for node, rank in zip(df['Node'], df['Rank'])}
    return rankings

# 文件列表
files = {
    'Degree': 'degree_sorted.csv',
    'Closeness': 'closeness_sorted.csv',
    'Betweenness': 'betweenness_sorted.csv',
    'Eigenvector': 'eigenvector_sorted.csv',
    'CT-PR': 'custom_sorted.csv'

}

# 计算每种方法的单调性函数
node_count = len(pd.read_csv('degree_sorted.csv'))  # 假设所有文件中的节点数量相同

monotonicity_results = {}
for method, filename in files.items():
    rankings = load_and_rank(filename)
    monotonicity = compute_monotonicity({method: rankings}, node_count)
    monotonicity_results[method] = monotonicity

# 打印单调性函数结果
for method, value in monotonicity_results.items():
    print(f"Method: {method}")
    print(f"Monotonicity Value: {value}\n")